In [ ]:
##################################################################################################
### 
###  Per data set, 2016-2019, 
###               create a clean gender column
###               eliminate error ages
###               create bins for ages
###               fillna
###
###  Add a 'year' column to each, concatenate all four dataframes, and add a unique individual ID column
###
###  Save combined dataframe to '/Resources/combined_df.csv'
###
###################################################################################################

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from pprint import pprint
from pathlib import Path
import numpy as np

# File locations
cleaned_2016 = "Resources/2016-cleaned.csv"
survey_2014 = "Resources/2014-survey.csv"
survey_2016 = "Resources/2016-survey.csv"
survey_2017 = "Resources/2017-survey.csv"
survey_2018 = "Resources/2018-survey.csv"
survey_2019 = "Resources/2019-survey.csv"

# Reading the data
clean_2016 = pd.read_csv(cleaned_2016)
s_2014_data = pd.read_csv(survey_2014)
s_2016_data = pd.read_csv(survey_2016)
s_2017_data = pd.read_csv(survey_2017)
s_2018_data = pd.read_csv(survey_2018)
s_2019_data = pd.read_csv(survey_2019)

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
###################################################################################################
###  Pre Work
###
###  Set up variables for cleaning and create functions to be used by all four data sets, 2016-2019
###
###################################################################################################

In [4]:
############ Identify the columns for each data set year

In [5]:
column_list_2016 = ['What is your age?', 
                    'What is your gender?',
                    'What country do you work in?', 
                    'How many employees does your company or organization have?',
                    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
                    'Have you had a mental health disorder in the past?',
                    'Do you currently have a mental health disorder?',
                    'Would you be willing to bring up a physical health issue with a potential employer in an interview?',
                    'Would you bring up a mental health issue with a potential employer in an interview?',
                    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'
                   ]

In [6]:
column_list_2017 = ['What is your age?', 
                    'What is your gender?', 
                    'What country do you <strong>work</strong> in?', 
                    'How many employees does your company or organization have?', 
                    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
                    'Have you had a mental health disorder in the past?',
                    'Do you currently have a mental health disorder?',
                    'Would you be willing to bring up a physical health issue with a potential employer in an interview?', 
                    'Would you bring up your mental health with a potential employer in an interview?', 
                    'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?' 
                    ]

In [7]:
column_list_2018 = column_list_2017

In [8]:
column_list_2019 = ['What is your age?', 
               'What is your gender?', 
               'What country do you *work* in?', 
               'How many employees does your company or organization have?', 
               'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
               'Have you had a mental health disorder in the past?',
               'Do you *currently* have a mental health disorder?',
               'Would you be willing to bring up a physical health issue with a potential employer in an interview?', 
               'Would you bring up your *mental* health with a potential employer in an interview?', 
               'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?' 
               ]

In [9]:
######## function to narrow and rename columns in every data set

def narrow(df, columns):
    narrowed_df = df[columns]
    new_column_names = ['age',  
            'original_gender',  
            'country',  
            'num_employees',
            'employer_discuss_mh',
            'disorder_past',  
            'disorder_current', 
            'bring_up_phys_issue_in_interview', 
            'bring_up_mh_issue_in_interview', 
            'request_med_leave']
    narrowed_df.columns = new_column_names
    return narrowed_df

In [10]:
###############################
### prepare for gender work
###############################

In [11]:
# begin with 2016 gender lists
running_male_list = ['Male',
             'male',
             'Male ',
             'M',
             'm',
             'man',
             'Cis male',
             'Male.',
             'Male (cis)',
             'Man',
             'Sex is male',
             'cis male',
             'Malr',
             'Dude', 
             "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
             'mail',
             'male ',
             'Cis Male',
             'cisdude',
             'cis man', 
             'MALE']

running_female_list =['Female', 
              'female', 
              'I identify as female.', 
              'female ',
              'Female assigned at birth ', 
              'F', 
              'Woman', 
              'f',
              'Cis female ', 
              'Female ',
              'woman', 
              'female/woman',
              'Cisgender Female', 
              'fem', 
              'Female (props for making this a freeform field, though)',
              ' Female', 
              'Cis-woman', 
              'female-bodied; no feelings about gender', 
              'AFAB',
              'fm'  ]

running_other_gender_list = ['Bigender',  
                     'non-binary',  
                     'Transitioned, M2F',  
                     'Genderfluid (born female)',  
                     'Other/Transfeminine', 
                     'Female or Multi-Gender Femme',  
                     'Androgynous',
                     'male 9:1 female, roughly', 
                     'Other', 
                     'nb masculine',  
                     'genderqueer', 
                     'Human', 
                     'Genderfluid', 
                     'Enby', 
                     'genderqueer woman', 
                     'mtf', 
                     'Queer',
                     'Agender', 
                     'Fluid', 
                     'Male/genderqueer', 
                     'Nonbinary', 
                     'human', 
                     'Unicorn', 
                     'Male (trans, FtM)', 
                     'Genderqueer', 
                     'Genderflux demi-girl',
                     'Transgender woman']

running_no_answer_list = ['nan', 'none of your business',  'M|']

#combine lists to create one for comparison and to grow
#running_combined_gender_list = running_male_list + running_female_list + running_other_gender_list + running_no_answer_list
#combined_gender_list

In [12]:
########## function to find unique gender values

def find_new_genders(df):
    
    # list unique values from free-form answers
    unique_genders = df["original_gender"].unique()
    
    ## find unique responses in df
    different_responses = []
    for gender in unique_genders:
        if gender not in running_combined_gender_list:
            different_responses.append(gender)
    return different_responses
    

In [13]:
############### function to print counts of new lists and print messages

def check_gender_counts(different_responses, add_male, add_female, add_other, add_no):
     
    # check that all values are accounted for with counts
    print(f"There are", len(different_responses), "unique original gender values")
    print(f"There are", len(add_male), "unique male values")
    print(f"There are", len(add_female), "unique female values")
    print(f"There are", len(add_other), "unique other gender values")
    print(f"There are", len(add_no), "unique no answer values")
    
    if len(different_responses) != len(add_male) + len(add_female) + len(add_other) + len(add_no):
        print('Counts do not add up - check gender lists')
    else:
        print("Counts look good")


In [14]:
############## function to append new unique values to running lists and then create new column and fill

def create_new_column(df, male, female, other, no):
    
    # append male, female, other, and no answer lists
    for i in male:
        running_male_list.append(i)
    for i in female:
        running_female_list.append(i)
    for i in other:
        running_other_gender_list.append(i)
    for i in no:
        running_no_answer_list.append(i)

    # create new column called 'gender' and assign values based on the gender categories
    df.insert(loc=2, column='gender', value='')
    
    # create a list for new gender values
    new_gender_list = []
    for gender in df["original_gender"]:
        if gender in running_male_list:
            new_gender_list.append("male")
        elif gender in running_female_list:
            new_gender_list.append("female")
        elif gender in running_other_gender_list:
            new_gender_list.append("other")
        else:
            new_gender_list.append("")       
    
    # copy df and assign values to the gender column
    clean_df = pd.DataFrame(df).copy()
    clean_df['gender'] = new_gender_list
    
    return clean_df
    

In [15]:
############### Create bins and limits for ages cleaning
lower = 18
upper = 98
bins = [0,24,34,44,54,64,74,75]
age_groups = ["18-24","25-34","35-44","45-54","55-64","65-74","75+"]

In [16]:
################  function to fill blank or nan cells with "No Response"

def fill_df(df):
    return df.fillna(
        {
            "num_employees": 'No Response',
            "gender": 'No Response',
            "employer_discuss_mh": 'No Response',
            "disorder_past": 'No Response',
            "request_med_leave": 'No Response'
        }
    )

In [17]:
###################################################################################################
###  
###  Per data set, 2016-2019, 
###               create a clean gender column
###               eliminate error ages
###               create bins for ages
###               fillna
###
###################################################################################################

In [18]:
######################
###   2016
######################

In [19]:
# Call function to narrow and rename columns
narrowed_2016 = narrow(s_2016_data, column_list_2016)
narrowed_2016.head()

,age,original_gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,39,Male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy
1,29,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy
2,38,Male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult
3,43,male,United Kingdom,NaN,NaN,Yes,Yes,Yes,Maybe,NaN
4,43,Female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult


In [20]:
############### gender clean up 2016

In [21]:
# begin by updating the running gender list 
running_combined_gender_list = running_male_list + running_female_list + running_other_gender_list + running_no_answer_list

# call function to find the unique genders of this year's data set
different_responses_2016 = find_new_genders(narrowed_2016)
different_responses_2016

[nan]

In [22]:
# create lists to append to running lists based on the different_responses list 
add_male_list_2016 = []
add_female_list_2016 = []
add_other_gender_list_2016 = []
add_no_answer_list_2016 = ['nan']

# call function to check manual work to verify all new gender answers are covered then append to running lists
check_gender_counts(different_responses_2016, add_male_list_2016, add_female_list_2016, add_other_gender_list_2016, add_no_answer_list_2016)


There are 1 unique original gender values
There are 0 unique male values
There are 0 unique female values
There are 0 unique other gender values
There are 1 unique no answer values
Counts look good


In [23]:
#call function to append running lists, create new column and fill column with "male", "female", "other", or ""
clean_2016 = create_new_column(narrowed_2016, add_male_list_2016, add_female_list_2016, add_other_gender_list_2016, add_no_answer_list_2016)

print(clean_2016['gender'].unique())
clean_2016.head()     

['male' 'female' 'other' '']


,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,39,Male,male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy
1,29,male,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy
2,38,Male,male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult
3,43,male,male,United Kingdom,NaN,NaN,Yes,Yes,Yes,Maybe,NaN
4,43,Female,female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult


In [24]:
###########  age column clean up 2016

In [25]:
# age limits and bins set above
# Remove any ages below 98 and above 18
clean_2016 = clean_2016.loc[(clean_2016["age"] >= lower) & (clean_2016["age"] <= upper), :]
#clean_2016["age"].value_counts()

# Slice the data and place it into bins
age_groups_column = pd.cut(clean_2016["age"], bins, labels=age_groups)

#Add a new age groups column
clean_2016["age groups"] = age_groups_column
clean_2016.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups
0,39,Male,male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy,35-44
1,29,male,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy,25-34
2,38,Male,male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult,35-44
3,43,male,male,United Kingdom,NaN,NaN,Yes,Yes,Yes,Maybe,NaN,35-44
4,43,Female,female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult,35-44


In [26]:
# call function to fill blank or nan cells with "No Response"
clean_2016 = fill_df(clean_2016)
clean_2016.count()

age                                 1428
original_gender                     1425
gender                              1428
country                             1428
num_employees                       1428
employer_discuss_mh                 1428
disorder_past                       1428
disorder_current                    1428
bring_up_phys_issue_in_interview    1428
bring_up_mh_issue_in_interview      1428
request_med_leave                   1428
age groups                          1428
dtype: int64

In [27]:
######################
###   2017
######################

In [28]:
# Call function to narrow and rename columns
narrowed_2017 = narrow(s_2017_data, column_list_2017)
narrowed_2017.head()

,age,original_gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,27.0,Female,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know
1,31.0,male,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know
2,36.0,male,United States of America,6-25,I don't know,Yes,Yes,Maybe,No,Difficult
3,22.0,Male,United States of America,More than 1000,I don't know,No,Yes,No,No,Difficult
4,52.0,female,United States of America,NaN,NaN,Yes,No,Maybe,No,NaN


In [29]:
############### gender clean up 2017

In [30]:
# begin by updating the running gender list 
running_combined_gender_list = running_male_list + running_female_list + running_other_gender_list + running_no_answer_list

# call function to find the unique genders of this year's data set
different_responses_2017 = find_new_genders(narrowed_2017)
different_responses_2017

[nan,
 'cis-male',
 'Mail',
 'male/androgynous ',
 'cis hetero male',
 'femalw',
 'femail',
 'uhhhhhhhhh fem genderqueer?',
 "male (hey this is the tech industry you're talking about)",
 'female (cis)',
 'God King of the Valajar',
 'My sex is female.',
 'Cis-male',
 'Non-binary',
 'Male, cis',
 'cis male ',
 'Agender/genderfluid',
 'Male-ish',
 'sometimes',
 'female (cisgender)',
 'Female (cis) ',
 'Woman-identified',
 'Contextual',
 'Non binary',
 'Genderqueer demigirl',
 'Genderqueer/non-binary',
 'cis-Female',
 'cis female',
 'nonbinary',
 'F, cisgender',
 'Female-ish',
 '\\-',
 'trans woman',
 'Transfeminine',
 'dude']

In [31]:
# create lists to append to running lists based on the different_responses list 
add_male_list_2017 = ['cis-male','Mail', 'cis hetero male',"male (hey this is the tech industry you're talking about)",'Cis-male','Male, cis',  'cis male ', 'dude']
add_female_list_2017 = ['femalw', 'femail', 'female (cis)','My sex is female.', 'female (cisgender)', 'Female (cis) ',  'Woman-identified', 'cis-Female', 'cis female', 'F, cisgender']
add_other_gender_list_2017 = ['male/androgynous ', 'uhhhhhhhhh fem genderqueer?', 'Non-binary', 'Agender/genderfluid',  'Male-ish', 'Contextual',  'Non binary',  'Genderqueer demigirl', 'Genderqueer/non-binary', 'nonbinary', 'Female-ish', 'trans woman', 'Transfeminine']
add_no_answer_list_2017 = ['God King of the Valajar', 'sometimes', '\\-', 'nan']

# call function to check manual work to verify all new gender answers are covered then append to running lists
check_gender_counts(different_responses_2017, add_male_list_2017, add_female_list_2017, add_other_gender_list_2017, add_no_answer_list_2017)


There are 35 unique original gender values
There are 8 unique male values
There are 10 unique female values
There are 13 unique other gender values
There are 4 unique no answer values
Counts look good


In [32]:
#call function to append running lists, create new column and fill column with "male", "female", "other", or ""
clean_2017 = create_new_column(narrowed_2017, add_male_list_2017, add_female_list_2017, add_other_gender_list_2017, add_no_answer_list_2017)

print(clean_2017['gender'].unique())
clean_2017.head()     

['female' 'male' '' 'other']


,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,27.0,Female,female,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know
1,31.0,male,male,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know
2,36.0,male,male,United States of America,6-25,I don't know,Yes,Yes,Maybe,No,Difficult
3,22.0,Male,male,United States of America,More than 1000,I don't know,No,Yes,No,No,Difficult
4,52.0,female,female,United States of America,NaN,NaN,Yes,No,Maybe,No,NaN


In [33]:
###########  age column clean up 2017

In [34]:
# age limits and bins set above
# Remove any ages below 98 and above 18
clean_2017 = clean_2017.loc[(clean_2017["age"] >= lower) & (clean_2017["age"] <= upper), :]
#clean_2017["age"].value_counts()

# Slice the data and place it into bins
age_groups_column = pd.cut(clean_2017["age"], bins, labels=age_groups)

#Add a new age groups column
clean_2017["age groups"] = age_groups_column
clean_2017.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups
0,27.0,Female,female,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know,25-34
1,31.0,male,male,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know,25-34
2,36.0,male,male,United States of America,6-25,I don't know,Yes,Yes,Maybe,No,Difficult,35-44
3,22.0,Male,male,United States of America,More than 1000,I don't know,No,Yes,No,No,Difficult,18-24
4,52.0,female,female,United States of America,NaN,NaN,Yes,No,Maybe,No,NaN,45-54


In [35]:
# call function to fill blank or nan cells with "No Response"
clean_2017 = fill_df(clean_2017)
clean_2017.count()

age                                 754
original_gender                     742
gender                              754
country                             754
num_employees                       754
employer_discuss_mh                 754
disorder_past                       754
disorder_current                    754
bring_up_phys_issue_in_interview    754
bring_up_mh_issue_in_interview      754
request_med_leave                   754
age groups                          754
dtype: int64

In [36]:
######################
###   2018
######################

In [37]:
# Call function to narrow and rename columns
narrowed_2018 = narrow(s_2018_data, column_list_2018)
narrowed_2018.head()

,age,original_gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,57,Female,Canada,More than 1000,Yes,Yes,Possibly,No,No,Somewhat difficult
1,29,male,United States of America,More than 1000,No,Yes,Yes,Yes,No,Somewhat difficult
2,46,Male,United States of America,6-25,No,No,No,No,No,Somewhat easy
3,34,male,Norway,6-25,No,No,No,No,No,Neither easy nor difficult
4,29,Ostensibly Male,United States of America,26-100,Yes,Yes,Yes,Yes,Yes,Somewhat easy


In [38]:
############### gender clean up 2018

In [39]:
# begin by updating the running gender list 
running_combined_gender_list = running_male_list + running_female_list + running_other_gender_list + running_no_answer_list

# call function to find the unique genders of this year's data set
different_responses_2018 = find_new_genders(narrowed_2018)
different_responses_2018

['Ostensibly Male',
 'male, born with xy chromosoms',
 'Malel',
 nan,
 'I identify as female',
 '*shrug emoji* (F)',
 'Male (or female, or both)',
 'Trans man',
 'transgender',
 'non binary',
 'Female/gender non-binary.',
 'Cis woman',
 'Female (cisgender)',
 'Cis-Female',
 'Cisgendered woman',
 'Trans woman',
 'genderfluid',
 'Demiguy',
 'none',
 'Trans female',
 'Cisgender male',
 'She/her/they/them',
 'SWM',
 'cisgender female',
 'NB',
 'Nonbinary/femme',
 'gender non-conforming woman']

In [40]:
# create lists to append to running lists based on the different_responses list 
add_male_list_2018 = ['Ostensibly Male', 'male, born with xy chromosoms', 'Malel', 'Cis-male','Cisgender male', 'SWM']
add_female_list_2018 = ['I identify as female', '*shrug emoji* (F)', 'Cis woman', 'Female (cisgender)', 'Cis-Female', 'Cisgendered woman', 'cisgender female']
add_other_gender_list_2018 = ['Male (or female, or both)', 'Trans man','transgender', 'non binary', 'Female/gender non-binary.','She/her/they/them','NB', 'Nonbinary/femme', 'gender non-conforming woman', 'Trans woman', 'genderfluid', 'Demiguy', 'none', 'Trans female']
add_no_answer_list_2018 = ['nan']

# call function to check manual work to verify all new gender answers are covered then append to running lists
check_gender_counts(different_responses_2018, add_male_list_2018, add_female_list_2018, add_other_gender_list_2018, add_no_answer_list_2018)


There are 27 unique original gender values
There are 6 unique male values
There are 7 unique female values
There are 14 unique other gender values
There are 1 unique no answer values
Counts do not add up - check gender lists


In [41]:
#call function to append running lists, create new column and fill column with "male", "female", "other", or ""
clean_2018 = create_new_column(narrowed_2018, add_male_list_2018, add_female_list_2018, add_other_gender_list_2018, add_no_answer_list_2018)

print(clean_2018['gender'].unique())
clean_2018.head()     

['female' 'male' 'other' '']


,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,57,Female,female,Canada,More than 1000,Yes,Yes,Possibly,No,No,Somewhat difficult
1,29,male,male,United States of America,More than 1000,No,Yes,Yes,Yes,No,Somewhat difficult
2,46,Male,male,United States of America,6-25,No,No,No,No,No,Somewhat easy
3,34,male,male,Norway,6-25,No,No,No,No,No,Neither easy nor difficult
4,29,Ostensibly Male,male,United States of America,26-100,Yes,Yes,Yes,Yes,Yes,Somewhat easy


In [42]:
###########  age column clean up 2018

In [43]:
# age limits and bins set above
# Remove any ages below 98 and above 18
clean_2018 = clean_2018.loc[(clean_2018["age"] >= lower) & (clean_2018["age"] <= upper), :]
#clean_2018["age"].value_counts()

# Slice the data and place it into bins
age_groups_column = pd.cut(clean_2018["age"], bins, labels=age_groups)

#Add a new age groups column
clean_2018["age groups"] = age_groups_column
clean_2018.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups
0,57,Female,female,Canada,More than 1000,Yes,Yes,Possibly,No,No,Somewhat difficult,55-64
1,29,male,male,United States of America,More than 1000,No,Yes,Yes,Yes,No,Somewhat difficult,25-34
2,46,Male,male,United States of America,6-25,No,No,No,No,No,Somewhat easy,45-54
3,34,male,male,Norway,6-25,No,No,No,No,No,Neither easy nor difficult,25-34
4,29,Ostensibly Male,male,United States of America,26-100,Yes,Yes,Yes,Yes,Yes,Somewhat easy,25-34


In [44]:
# call function to fill blank or nan cells with "No Response"
clean_2018 = fill_df(clean_2018)
clean_2018.count()

age                                 417
original_gender                     414
gender                              417
country                             417
num_employees                       417
employer_discuss_mh                 417
disorder_past                       417
disorder_current                    417
bring_up_phys_issue_in_interview    417
bring_up_mh_issue_in_interview      417
request_med_leave                   417
age groups                          417
dtype: int64

In [45]:
######################
###   2019
######################

In [46]:
# Call function to narrow and rename columns
narrowed_2019 = narrow(s_2019_data, column_list_2019)
narrowed_2019.head()

,age,original_gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,25,Male,United States of America,26-100,Yes,No,Don't Know,Yes,Maybe,Very easy
1,51,male,United States of America,26-100,No,Possibly,Possibly,No,No,I don't know
2,27,Male,United States of America,26-100,No,No,No,Maybe,No,Somewhat difficult
3,37,male,United States of America,100-500,Yes,No,No,No,No,Very easy
4,46,m,United States of America,26-100,I don't know,No,No,No,No,I don't know


In [47]:
############### gender clean up 2019

In [48]:
# begin by updating the running gender list 
running_combined_gender_list = running_male_list + running_female_list + running_other_gender_list + running_no_answer_list

# call function to find the unique genders of this year's data set
different_responses_2019 = find_new_genders(narrowed_2019)
different_responses_2019

[nan,
 'Let\'s keep it simple and say "male"',
 'Identify as male',
 'Masculine',
 'Cishet male',
 'Female-identified',
 'agender',
 'Questioning',
 'I have a penis',
 'rr',
 'cis woman',
 'Agender trans woman',
 'femmina',
 '43',
 'masculino',
 'I am a Wookie',
 'Make',
 'Trans non-binary/genderfluid',
 'CIS Male',
 'Non-binary and gender fluid',
 'Femile',
 'Female (cis)']

In [49]:
# create lists to append to running lists based on the different_responses list 
add_male_list_2019 = ['Ostensibly Male', 'male, born with xy chromosoms', 'Malel', 'Cis-male','Cisgender male', 'SWM']
add_female_list_2019 = ['I identify as female', '*shrug emoji* (F)', 'Cis woman', 'Female (cisgender)', 'Cis-Female', 'Cisgendered woman', 'cisgender female']
add_other_gender_list_2019 = ['Male (or female, or both)', 'Trans man','transgender', 'non binary', 'Female/gender non-binary.','She/her/they/them','NB', 'Nonbinary/femme', 'gender non-conforming woman', 'Trans woman', 'genderfluid', 'Demiguy', 'none', 'Trans female']
add_no_answer_list_2019 = ['nan']

# call function to check manual work to verify all new gender answers are covered then append to running lists
check_gender_counts(different_responses_2019, add_male_list_2019, add_female_list_2019, add_other_gender_list_2019, add_no_answer_list_2019)


There are 22 unique original gender values
There are 6 unique male values
There are 7 unique female values
There are 14 unique other gender values
There are 1 unique no answer values
Counts do not add up - check gender lists


In [50]:
#call function to append running lists, create new column and fill column with "male", "female", "other", or ""
clean_2019 = create_new_column(narrowed_2019, add_male_list_2019, add_female_list_2019, add_other_gender_list_2019, add_no_answer_list_2019)

print(clean_2019['gender'].unique())
clean_2019.head()     

['male' 'female' '' 'other']


,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave
0,25,Male,male,United States of America,26-100,Yes,No,Don't Know,Yes,Maybe,Very easy
1,51,male,male,United States of America,26-100,No,Possibly,Possibly,No,No,I don't know
2,27,Male,male,United States of America,26-100,No,No,No,Maybe,No,Somewhat difficult
3,37,male,male,United States of America,100-500,Yes,No,No,No,No,Very easy
4,46,m,male,United States of America,26-100,I don't know,No,No,No,No,I don't know


In [51]:
###########  age column clean up 2019

In [52]:
# age limits and bins set above
# Remove any ages below 98 and above 18
clean_2019 = clean_2019.loc[(clean_2019["age"] >= lower) & (clean_2019["age"] <= upper), :]
#clean_2019["age"].value_counts()

# Slice the data and place it into bins
age_groups_column = pd.cut(clean_2019["age"], bins, labels=age_groups)

#Add a new age groups column
clean_2019["age groups"] = age_groups_column
clean_2019.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups
0,25,Male,male,United States of America,26-100,Yes,No,Don't Know,Yes,Maybe,Very easy,25-34
1,51,male,male,United States of America,26-100,No,Possibly,Possibly,No,No,I don't know,45-54
2,27,Male,male,United States of America,26-100,No,No,No,Maybe,No,Somewhat difficult,25-34
3,37,male,male,United States of America,100-500,Yes,No,No,No,No,Very easy,35-44
4,46,m,male,United States of America,26-100,I don't know,No,No,No,No,I don't know,45-54


In [53]:
# call function to fill blank or nan cells with "No Response"
clean_2019 = fill_df(clean_2019)
clean_2019.count()

age                                 351
original_gender                     344
gender                              351
country                             351
num_employees                       351
employer_discuss_mh                 351
disorder_past                       351
disorder_current                    351
bring_up_phys_issue_in_interview    351
bring_up_mh_issue_in_interview      351
request_med_leave                   351
age groups                          351
dtype: int64

In [54]:
###################################################################################################
### 
###   make timeline df for each year, add year column, then concatenate
###
###################################################################################################

In [55]:
timeline_2016 = clean_2016.copy()
timeline_2016["year"] = 2016
timeline_2016.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year
0,39,Male,male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy,35-44,2016
1,29,male,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy,25-34,2016
2,38,Male,male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult,35-44,2016
3,43,male,male,United Kingdom,No Response,No Response,Yes,Yes,Yes,Maybe,No Response,35-44,2016
4,43,Female,female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult,35-44,2016


In [56]:
timeline_2017 = clean_2017.copy()
timeline_2017["year"] = 2017
timeline_2017.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year
0,27.0,Female,female,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know,25-34,2017
1,31.0,male,male,United Kingdom,100-500,No,Possibly,Possibly,Yes,No,I don't know,25-34,2017
2,36.0,male,male,United States of America,6-25,I don't know,Yes,Yes,Maybe,No,Difficult,35-44,2017
3,22.0,Male,male,United States of America,More than 1000,I don't know,No,Yes,No,No,Difficult,18-24,2017
4,52.0,female,female,United States of America,No Response,No Response,Yes,No,Maybe,No,No Response,45-54,2017


In [57]:
timeline_2018 = clean_2018.copy()
timeline_2018["year"] = 2018
timeline_2018.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year
0,57,Female,female,Canada,More than 1000,Yes,Yes,Possibly,No,No,Somewhat difficult,55-64,2018
1,29,male,male,United States of America,More than 1000,No,Yes,Yes,Yes,No,Somewhat difficult,25-34,2018
2,46,Male,male,United States of America,6-25,No,No,No,No,No,Somewhat easy,45-54,2018
3,34,male,male,Norway,6-25,No,No,No,No,No,Neither easy nor difficult,25-34,2018
4,29,Ostensibly Male,male,United States of America,26-100,Yes,Yes,Yes,Yes,Yes,Somewhat easy,25-34,2018


In [58]:
timeline_2019 = clean_2019.copy()
timeline_2019["year"] = 2019
timeline_2019.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year
0,25,Male,male,United States of America,26-100,Yes,No,Don't Know,Yes,Maybe,Very easy,25-34,2019
1,51,male,male,United States of America,26-100,No,Possibly,Possibly,No,No,I don't know,45-54,2019
2,27,Male,male,United States of America,26-100,No,No,No,Maybe,No,Somewhat difficult,25-34,2019
3,37,male,male,United States of America,100-500,Yes,No,No,No,No,Very easy,35-44,2019
4,46,m,male,United States of America,26-100,I don't know,No,No,No,No,I don't know,45-54,2019


In [59]:
combined_df = pd.concat([timeline_2016, timeline_2017, timeline_2018, timeline_2019],axis=0 , ignore_index= True)
combined_df.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year
0,39.0,Male,male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy,35-44,2016
1,29.0,male,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy,25-34,2016
2,38.0,Male,male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult,35-44,2016
3,43.0,male,male,United Kingdom,No Response,No Response,Yes,Yes,Yes,Maybe,No Response,35-44,2016
4,43.0,Female,female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult,35-44,2016


In [60]:
individual_list = np.arange(1, len(combined_df) + 1, 1)
combined_df['individual'] = individual_list
combined_df.head()

,age,original_gender,gender,country,num_employees,employer_discuss_mh,disorder_past,disorder_current,bring_up_phys_issue_in_interview,bring_up_mh_issue_in_interview,request_med_leave,age groups,year,individual
0,39.0,Male,male,United Kingdom,26-100,No,Yes,No,Maybe,Maybe,Very easy,35-44,2016,1
1,29.0,male,male,United States of America,6-25,Yes,Yes,Yes,Maybe,No,Somewhat easy,25-34,2016,2
2,38.0,Male,male,United Kingdom,6-25,No,Maybe,No,Yes,Yes,Neither easy nor difficult,35-44,2016,3
3,43.0,male,male,United Kingdom,No Response,No Response,Yes,Yes,Yes,Maybe,No Response,35-44,2016,4
4,43.0,Female,female,United States of America,6-25,No,Yes,Yes,Maybe,No,Neither easy nor difficult,35-44,2016,5


In [61]:
combined_df['year'].value_counts()

year
2016    1428
2017     754
2018     417
2019     351
Name: count, dtype: int64

In [62]:
# export time_line_df as a csv file
filepath = Path('Resources/combined_df.csv')
combined_df.to_csv(filepath)